In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from flask import Flask, render_template, Response


In [2]:
app = Flask(__name__)


In [3]:
# Path to images for attendance
path = 'D:\Face-Recognition-Attendence-main\Mark Attendence Project\ImagesAttendence'

In [4]:
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['bapa.jpg', 'Bill Gates.jpg', 'Elon Musk.jpg', 'Jeff Bezos.jpg', 'maa.jpg', 'Mark Zuckerberg.jpg', 'mihir.JPG']


# Breakdown of the Code:

1. images = []:

This line creates an empty list named images. This list will be used to store image data as we process them.
2. classNames = []:

Similarly, this line creates an empty list named classNames. This list will store the names associated with each image.
3. myList = os.listdir(path):

This line uses the os.listdir(path) function to list all files and directories within the specified directory path.
The resulting list of filenames is stored in the myList variable.
Overall Purpose:

This code snippet is typically used as a preliminary step in many image processing and machine learning tasks. It:

Initializes data structures: Creates empty lists to store images and their corresponding class labels.
Lists files in a directory: Uses os.listdir to get a list of files within a specified directory.
Sets the stage for further processing: The myList containing filenames can be used to iterate over images, load them, and potentially preprocess them.
Example Usage:

In [ ]:
# Load the images and their corresponding class names
for i in myList:
    currentImage = cv2.imread(f'{path}/{i}')
    images.append(currentImage)
    classNames.append(os.path.splitext(i)[0])
print(classNames)

In [6]:
# Function to find and encode faces from the images
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        try:
            encode = face_recognition.face_encodings(img)[0]
            encodeList.append(encode)
        except IndexError:
            print("Face not found in image, skipping:", img)
            continue
    return encodeList

In [7]:
# Function to Mark Attendence
def markAttendece(name):
    with open("D:\Face-Recognition-Attendence-main\Mark Attendence Project\Attendance.csv",'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dateSrting = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dateSrting}')


In [8]:
# Get known face encodings
encodeListKnown = findEncodings(images)
print('Encoding completed!')

Encoding completed!


In [10]:
# Start the webcam and recognize faces in real-time
cap = cv2.VideoCapture(0)  # Assuming webcam index is 0

while True:
    success, img = cap.read()

    if not success:
        print("Failed to grab frame")
        break

    img = cv2.flip(img, 1)  # Flip the image horizontally (optional)

    # Resize frame for faster face recognition processing
    img_small = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    img_small = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)

    # Find all face locations and face encodings in the current frame
    faces_cur_frame = face_recognition.face_locations(img_small)
    encodings_cur_frame = face_recognition.face_encodings(img_small, faces_cur_frame)

    # Loop through each face found in the frame
    for encode_face, face_loc in zip(encodings_cur_frame, faces_cur_frame):
        matches = face_recognition.compare_faces(encodeListKnown, encode_face)
        face_dis = face_recognition.face_distance(encodeListKnown, encode_face)

        # Find the best match
        match_index = np.argmin(face_dis)

        if matches[match_index]:
            name = classNames[match_index].upper()

            # Scale the face location back up since we resized the frame
            top, right, bottom, left = face_loc
            top, right, bottom, left = top * 4, right * 4, bottom * 4, left * 4

            # Draw a rectangle around the face
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)

            # Draw the name of the person
            cv2.putText(img, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

            # Call your markAttendance function here
            markAttendece(name)

    cv2.imshow('Webcam', img)

    # Exit on pressing 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()